In [1]:
from PIL import Image
from numpy import asarray
from paz.pipelines import MiniXceptionFER

In [2]:
emotions = ["Anger", "Contempt", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Suprised"]

In [5]:
image = Image.open("./data/images/0/Anger.jpg")
print(image.format)
print(image.size)
print(image.mode)

numpy_image = asarray(image)
classify = MiniXceptionFER()
inference = classify(numpy_image)

inference

JPEG
(3880, 5184)
RGB


{'class_name': 'neutral',
 'scores': array([[0.13247669, 0.00651476, 0.01112644, 0.35317189, 0.12061383,
         0.00324964, 0.37284675]], dtype=float32)}